In [1]:
import sys
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import time
import pytz
import pandas as pd
import json 
import re


In [2]:
url='https://speakday.com/timetable/english-conversations'


In [3]:
def test_print(url):
    print(url)

In [4]:
def get_page(url):  #be careful, we don't want to crash the site
    HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Cache-Control": "max-age=0",
        }
    html = requests.get(url, headers=HEADERS)
    bs = BeautifulSoup(html.text, 'html.parser')
    return  bs

In [15]:


#filter data
def filter_speakday(bs):  
    script_json=bs.find('script', id='__NEXT_DATA__')
    json_object = json.loads(script_json.contents[0])
    list_seesion_big=json_object['props']['pageProps']['timetableData']['conversations']
    list_seesion=[]
    for session in list_seesion_big:
        for i in session:
            if i in ('schedule'):
                schedule =(i,session[i]['date_time'])
        dict_small=dict([schedule])
        list_seesion.append(dict_small)
    df_import = pd.DataFrame.from_records(list_seesion)
    

    df_import['schedule']=pd.to_datetime(df_import['schedule']).dt.tz_convert('Europe/Kyiv')
    
    #get current dateTime
    current_dateTime = pd.to_datetime('today').tz_localize('Europe/Kyiv')
    #end time session only 20 minutes
    time_change = timedelta(minutes=20)



    #filter by upcoming events only
    df_not_group= df_import[df_import['schedule'] >= current_dateTime].copy()



    #grouping
    df_not_group = df_not_group.rename(columns={'schedule': 'start_event'})
    df_not_group['end_event']=df_not_group['start_event']+time_change
    
    df_not_group=df_not_group.sort_values(by='start_event')
    df_upcoming = df_not_group.groupby(
        (df_not_group["end_event"].diff() != pd.Timedelta("20 minutes")).cumsum(), as_index=False
        ).agg({"start_event": "first", "end_event": "last"})
    
    

    #change time fromat
    df_upcoming["start_event"] = df_upcoming["start_event"].apply(lambda x: x.isoformat())
    df_upcoming["end_event"] = df_upcoming["end_event"].apply(lambda x: x.isoformat())
    
    #final table
    df_upcoming['event_name']='Speakday'
    df_upcoming['description']=url
    df_upcoming['location']=None
    df_upcoming=df_upcoming[['event_name', 'description',  'start_event', 'end_event', 'location']]

    
    return df_upcoming



In [6]:
#be careful, we don't want to crash the site
#bs_speakday=get_page(url)

In [12]:
#bs_speakday

In [16]:
#df_upcoming_speakday=filter_speakday(bs_speakday)

In [18]:
#df_upcoming_speakday